# Data collection with the help of SerpAPI

Real time job details have been collected for various Data Science positions accross the United States.

In [ ]:
!pip install serpapi
pip install google-search-results

In [ ]:
from serpapi import GoogleSearch
#from dataExtraction import extract
from IPython.display import clear_output
from pprint import pprint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from serpapi import GoogleSearch
import csv
from pprint import pprint  # For debugging
import requests  # We will use this to fetch the next page directly

# API key and country setup
api_key = "cc95ab69c225070c373d9c729ed84c365acfe7b37d6e8207b765a95462fc12f4"
country = "us"  # You can switch this to another valid country code

# Target number of job listings to collect (11,000 rows)
target_rows = 11000

# Function to extract data from results and write to CSV
def extract_to_csv(results, csv_writer, total_rows_collected):
    if 'jobs_results' in results and results['jobs_results']:
        for job in results['jobs_results']:
            # Stop when the target number of rows is reached
            if total_rows_collected >= target_rows:
                return total_rows_collected

            # Extract relevant job information
            title = job.get('title', 'N/A')
            company_name = job.get('company_name', 'N/A')
            location = job.get('location', 'N/A')
            via = job.get('via', 'N/A')
            description = job.get('description', 'N/A')
            job_highlights = ', '.join([item for highlight in job.get('job_highlights', []) for item in highlight.get('items', [])])
            detected_extensions = job.get('detected_extensions', {})
            date_posted = detected_extensions.get('posted_at', 'N/A')
            schedule_type = detected_extensions.get('schedule_type', 'N/A')
            job_id = job.get('job_id', 'N/A')
            share_link = job.get('share_link', 'N/A')

            # Write the job information as a row to the CSV file
            csv_writer.writerow([title, company_name, location, via, description, job_highlights, date_posted, schedule_type, job_id, share_link])

            total_rows_collected += 1  # Keep track of rows collected
    else:
        print("No job listings found in the API response.")

    return total_rows_collected

# Open a CSV file to write the data
with open('job_results_Lead Data Scientist.csv', mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    # Write the header row
    csv_writer.writerow(['Title', 'Company Name', 'Location', 'Via', 'Description', 'Job Highlights', 'Date Posted', 'Schedule Type', 'Job ID', 'Share Link'])

    # Initial search parameters
    params = {
        "engine": "google_jobs",
        "q": "Lead Data Scientist",  # Search query
        "hl": "en",
        "api_key": api_key,
        "gl": country
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    total_rows_collected = 0  # Keep track of how many rows have been collected
    pages_collected = 0  # Keep track of how many pages have been collected

    # Loop through pages using `next_page_token` and stop when target rows are collected
    while total_rows_collected < target_rows and 'jobs_results' in results:
        # Extract the current page's data
        total_rows_collected = extract_to_csv(results, csv_writer, total_rows_collected)

        # Debug: Print current rows collected
        print(f"Total rows collected so far: {total_rows_collected}")

        # Check if there's a next page token for pagination
        if 'serpapi_pagination' in results and 'next' in results['serpapi_pagination']:
            next_page_url = results['serpapi_pagination']['next']

            # Debug: Print the next page URL
            print(f"Next page URL: {next_page_url}")

            # Append the API key to the next page URL
            next_page_url_with_key = f"{next_page_url}&api_key={api_key}"

            # Fetch the next page using the full URL with the API key
            response = requests.get(next_page_url_with_key)
            results = response.json()  # Convert response to dictionary format
            pages_collected += 1  # Increment the page count
        else:
            print("No next page token found, stopping pagination.")
            break  # Exit loop if there are no more pages

# Final output
print(f"{pages_collected} pages of job listings collected.")
print(f"{total_rows_collected} job listings have been successfully written to 'job_results.csv'.")


Total rows collected so far: 10
Next page URL: https://serpapi.com/search.json?engine=google_jobs&gl=us&google_domain=google.com&hl=en&next_page_token=eyJmYyI6IkVxSURDdUlDUVVwSE9VcHJUbmhCV0RWR1kwNUdkMHR5WmxWU1preEpjRVJLTFhkUGVURlNjVzkyY2pOMmVrWTFTR2xpVG5WRVpYRXhRMlpaYld0V1FXNXJiVWMzV1ZwVVpVaFlYelp1VTNjMU5qaFFVWHBqU210TWNUVnhNVkJ0Y25wRmJscFBiRU4xTVhseE1IUldkMGg1YVdaR01FcHFjelJyZG5wUWNERnZXQzFTVUZCa00wOUlRV0kyVlZKTVYyOVlVbkJaTW5GT1RtSkdiRmxKWlU1SVdYVnFhbWc1VGpadVdtcEViR1pNVDFoQlRVOHhOemxmT1Vab1RERjRVM05TWTNsSmNGbEZTR0ZXTUhad1QzSXhVMlI2TkUxMVJVZFJhbGRtU1c5U1JFVk1RWEJhYms0NWRGOVlWazFCV2s0MmFGZEplbVJxYTBkR1pqZEpRWFJCU25GVllUbE5UVEoxUWkxNWJUWnFiVzF2YzFsUFNYVlZhSFpJVm5wWmFtVmhURkl0T0Y5U2MweHpWVjlrVFRGVFNYRm5jVTFsWlc0NVpYcEJkR3d4VFZwVWVWSkNaVloxTlVSNFRWaElMVU5WZVdGNVdra3paa0l6YmpKcWJFNWZReTFrV1hOM0VoZGxaR2N6V2pkbFVFSlpTMGhvWWtsUWRtWXpXVFJSZHhvaVFVWllja1ZqY0UwME9HWmFjMngwZG5kQ1NYTm1PR0o0UVU1cFRYcHNlVGRMUVEiLCJmY3YiOiIzIn0%3D&q=Lead+Data+Scientist
Total rows collected so far: 20
Next page URL: h